In [ ]:
!pip install -qU openai-agents==0.0.3

First let's set our [OpenAI API key](https://platform.openai.com/settings/organization/api-keys).

In [1]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or \
  getpass("Enter your OpenAI API key: ")

In [2]:
from agents import Agent, Runner
from openai.types.responses import ResponseTextDeltaEvent

agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant",
    model="gpt-4o-mini",
)

In [3]:
result = await Runner.run(
    starting_agent=agent,
    input="tell me a short story"
)
result.final_output

"Once upon a time in a small village nestled between lush green hills, there lived a young girl named Lila. Lila had an extraordinary gift: she could speak to animals. While other children played with toys, Lila wandered the fields and forests, chatting with birds, rabbits, and even the wise old owl who perched on the ancient oak tree.\n\nOne sunny afternoon, while exploring the edge of the village, Lila heard a faint whimpering sound. Following the noise, she found a tiny fox trapped in a thicket of thorny bushes. The little creature's bright eyes sparkled with fear, and Lila's heart went out to it.\n\n“Don’t worry, I’m here to help,” she whispered gently. The fox looked at her, seemingly understanding, and calmed down as she carefully freed it from the thorns. Once free, the fox nuzzled Lila’s hand, grateful for her kindness.\n\nIn return, the fox led Lila to a hidden glade filled with shimmering flowers that glowed in the soft afternoon light. “This is the Secret Garden,” the fox wh

In [4]:
response = Runner.run_streamed(
    starting_agent=agent,
    input="hello there"
)
async for event in response.stream_events():
    print(event)

AgentUpdatedStreamEvent(new_agent=Agent(name='Assistant', instructions="You're a helpful assistant", handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], mcp_servers=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), type='agent_updated_stream_event')
RawResponsesStreamEvent(data=ResponseCreatedEvent(response=Response(id='resp_67ee87ae7db48192bbe3a039b81ea49a04fc9325808c570b', created_at=1743685550.0, error=None, incomplete_details=None, instructions="You're a helpful assistant", metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[], parallel_tool_calls=False, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reaso

We can filter these various event types to find only raw tokens like so:

## Tools

OpenAI included **function tools** as a key feature in their Agents SDK announcement. After turning everyone away from using _function calling_ to instead use _tool calling_, OpenAI have now decided that an LLM deciding to execute some code will be called _"function tools"_.

To use _function tools_ in Agents SDK we simply decorate a function with the `@function_tool` decorator like so:

In [5]:
from agents import function_tool
from typing import List, Union, Optional, Tuple

# @function_tool
# def multiply(x: float, y: float) -> float:
#     """Multiplies `x` and `y` to provide a precise
#     answer."""
#     return x*y


from typing import List, Optional, Union, Tuple

from typing import List, Optional, Union, Tuple, Dict

@function_tool
def build_query(
    select_columns: List[str],
    table_name: str,
    where_conditions: Optional[Union[
        Dict[str, Union[str, int, float]],
        List[List[Union[str, int, float]]]  # Changed from Tuple[...] to List[...]
    ]] = None,
    group_by_columns: Optional[List[str]] = None,
    having_conditions: Optional[Union[
        Dict[str, Union[str, int, float]],
        List[List[Union[str, int, float]]]  # Changed from Tuple[...] to List[...]
    ]] = None
) -> str:
    query = f"SELECT {', '.join(select_columns)} FROM {table_name}"
    
    # Process WHERE conditions
    if where_conditions:
        if isinstance(where_conditions, dict):
            conditions = " AND ".join(
                f"{col} = '{val}'" if isinstance(val, str) else f"{col} = {val}"
                for col, val in where_conditions.items()
            )
        else:  # List of tuples
            conditions = " AND ".join(
                f"{col} {op} '{val}'" if isinstance(val, str) else f"{col} {op} {val}"
                for col, op, val in where_conditions
            )
        query += f" WHERE {conditions}"
    
    # Process GROUP BY and HAVING conditions
    if group_by_columns:
        query += f" GROUP BY {', '.join(group_by_columns)}"
        if having_conditions:
            if isinstance(having_conditions, dict):
                conditions = " AND ".join(
                    f"{col} = '{val}'" if isinstance(val, str) else f"{col} = {val}"
                    for col, val in having_conditions.items()
                )
            else:
                conditions = " AND ".join(
                    f"{col} {op} '{val}'" if isinstance(val, str) else f"{col} {op} {val}"
                    for col, op, val in having_conditions
                )
            query += f" HAVING {conditions}"
    
    return query


Note that we have taken extra care to include a clear and descriptive function name, relatively clear parameter names, type annotations for both input parameters and expected output, and a natural language docstring that will be fed to the LLM and explain to it _what_ this tool does.

To run our agent _with_ tools we simply pass our new tool into the `tools` parameter during `Agent` initialization.

In [7]:
agent = Agent(
    name="Assistant",
    instructions=(
        "You're an experienced data analyse "
        "use the provided tools whenever possible. Do not "
        "rely on your own knowledge too much and instead "
        "use your tools to help you answer queries.",
        "I AM PROVIDING you with tool to retrive the data using retrieve relevent data IF REQUIRED this function"
        "contains parameters:-(select_columns, table_name, where_conditions=None, group_by_columns=None, having_conditions=None)"
        "NOTE THAT I WILL BE PROVIDING YOU THE DETAILS AND USE THIS FUNCTION  AND ADJUST THE PARAMETERS RESPECTIVELY BASED ON MY AS TO RETRIEVE THE DATA AND GIVE ME THE ANSWER"
    ),
    model="gpt-4o",
    tools=[build_query]  # note that we expect a list of tools
)

Now let's initialize a new runner and execute our agent with tools:

In [8]:

response = Runner.run_streamed(
    starting_agent=agent,
    input=" In table steps the step_id 22 was supposed to be executed before step_id 32, but the opposite happened this case could only happen when " \
    "custom_input column of step_id 25 is an empty list [], how do i figure this out using steps table how do i view that table also mention if you used the tool to determine or not"
)
        
async for event in response.stream_events():
    if event.type == "raw_response_event" and \
        isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Instructions must be a string or a function, got ("You're an experienced data analyse use the provided tools whenever possible. Do not rely on your own knowledge too much and instead use your tools to help you answer queries.", 'I AM PROVIDING you with tool to retrive the data using retrieve relevent data IF REQUIRED this functioncontains parameters:-(select_columns, table_name, where_conditions=None, group_by_columns=None, having_conditions=None)NOTE THAT I WILL BE PROVIDING YOU THE DETAILS AND USE THIS FUNCTION  AND ADJUST THE PARAMETERS RESPECTIVELY BASED ON MY AS TO RETRIEVE THE DATA AND GIVE ME THE ANSWER')
Instructions must be a string or a function, got ("You're an experienced data analyse use the provided tools whenever possible. Do not rely on your own knowledge too much and instead use your tools to help you answer queries.", 'I AM PROVIDING you with tool to retrive the data using retrieve relevent data IF REQUIRED this functioncontains parameters:-(select_columns, table_name

To determine if the issue with the execution order was due to the `custom_input` column being an empty list for `step_id` 25, I used a query to retrieve the `custom_input` for that specific step:

```sql
SELECT step_id, custom_input FROM steps WHERE step_id = 25;
```

You can run this query on your database to check the value of `custom_input`. If it is an empty list, it confirms the condition described.